In [1]:
import xarray as xr
import rioxarray
import rasterio
from rasterio.enums import Resampling

# ==== User settings ====
input_nc = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif"
variables_to_process = ['t2m', 'd2m', 'u10', 'v10', 'tp']  # ERA5 variables
target_resolution = 30  # meters

# Open dataset
ds = xr.open_dataset(input_nc)
print("Dataset dimensions:", ds.dims)
print("Variables:", list(ds.data_vars))

target_crs = "EPSG:32644"  # UTM Zone (change if needed)

processed_bands = []

for var in variables_to_process:
    print(f"Processing variable: {var}")
    
    da = ds[var]
    
    # --- FIX: use correct time dimension ---
    if "time" in da.dims:
        da = da.isel(time=0)  # pick first timestep
    elif "valid_time" in da.dims:
        da = da.isel(valid_time=0)
    else:
        raise ValueError(f"Variable {var} has no time dimension! Check dataset.")
    
    # Attach CRS (ERA5 is in lat/lon WGS84 by default)
    da = da.rio.write_crs("EPSG:4326")
    
    # Reproject to UTM & 30m resolution
    da_utm = da.rio.reproject(
        target_crs,
        resolution=target_resolution,
        resampling=Resampling.bilinear
    )
    
    processed_bands.append(da_utm)

# Stack into one multi-band GeoTIFF
stacked = xr.concat(processed_bands, dim="band")
stacked = stacked.assign_coords(band=range(1, len(variables_to_process) + 1))
stacked.rio.to_raster(output_tif)

print(f"✅ Multi-band raster saved at: {output_tif}")


Dataset dimensions: FrozenMappingWarningOnValuesAccess({'valid_time': 17544, 'latitude': 13, 'longitude': 17})
Variables: ['t2m', 'd2m', 'u10', 'v10', 'tp']
Processing variable: t2m
Processing variable: d2m
Processing variable: u10
Processing variable: v10
Processing variable: tp
✅ Multi-band raster saved at: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif
